In [ ]:
pip install thop


In [10]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import os
import pandas as pd
from thop import profile
import matplotlib.pyplot as plt

In [11]:

class Config:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.batch_size = 128
        self.epochs = 50
        self.learning_rate = 0.001
        self.num_classes = 10
        self.results_path = "results"
        self.model_save_path = os.path.join(self.results_path, "saved_models")
        self.log_save_path = os.path.join(self.results_path, "training_logs")
        os.makedirs(self.model_save_path, exist_ok=True)
        os.makedirs(self.log_save_path, exist_ok=True)

config = Config()
print(f"使用设备: {config.device}")
print(f"结果将保存在: '{config.results_path}' 文件夹下")


使用设备: cuda
结果将保存在: 'results' 文件夹下


In [12]:
# 数据加载与预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=config.batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=config.batch_size, shuffle=False, num_workers=2)



In [ ]:

# 4. 模型定义


class CustomBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_residual=False, use_bn=True):
        super(CustomBlock, self).__init__()
        self.use_residual = use_residual


        if self.use_residual and in_channels != out_channels:
            raise ValueError("残差连接要求输入和输出通道数相同")

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=not use_bn)
        self.bn1 = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=not use_bn)
        self.bn2 = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.use_residual:
            out += identity

        out = self.relu(out)
        return out


class CustomCNN(nn.Module):
    def __init__(self, use_residual=False, use_bn=True, num_classes=10):
        super(CustomCNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=not use_bn),
            nn.BatchNorm2d(64) if use_bn else nn.Identity(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.layer2 = CustomBlock(64, 64, use_residual, use_bn)
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=2, bias=not use_bn),
            nn.BatchNorm2d(128) if use_bn else nn.Identity(),
            nn.ReLU(inplace=True)
        )
        self.layer4 = CustomBlock(128, 128, use_residual, use_bn)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x



class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, use_bn=True):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(planes) if use_bn else nn.Identity()
        self.bn2 = nn.BatchNorm2d(planes) if use_bn else nn.Identity()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes) if use_bn else nn.Identity()
            )

    def forward(self, x, use_residual=True):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if use_residual:
            out += self.shortcut(x)
        out = torch.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, use_bn=True, use_residual=True):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.use_bn = use_bn
        self.use_residual = use_residual
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64) if self.use_bn else nn.Identity()
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s, self.use_bn))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        for layer_group in [self.layer1, self.layer2, self.layer3, self.layer4]:
            for layer in layer_group:
                out = layer(out, use_residual=self.use_residual)
        out = torch.nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(use_bn=True, use_residual=True):
    return ResNet(BasicBlock, [2, 2, 2, 2], use_bn=use_bn, use_residual=use_residual)

In [14]:

# 5. 训练和评估函数

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc="Training", unit="batch", leave=False)
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    return total_loss / len(dataloader)


def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Evaluating", unit="batch", leave=False)
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            progress_bar.set_postfix(acc=100. * correct / total)
    accuracy = 100. * correct / total
    avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy

In [15]:

# 6. 主执行逻辑
def run_experiment(model_name, model_fn):
    print(f"\n{'=' * 40}")
    print(f"开始实验: {model_name}")
    print(f"{'=' * 40}")

    model = model_fn().to(config.device)

    dummy_input = torch.randn(1, 3, 32, 32).to(config.device)
    flops, params = profile(model, inputs=(dummy_input,), verbose=False)
    params_m = params / 1e6
    flops_g = flops / 1e9
    print(f"模型: {model_name}")
    print(f"  -> 参数量 (Params): {params_m:.2f} M")
    print(f"  -> 计算量 (FLOPs): {flops_g:.2f} G")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.epochs)

    start_time = time.time()
    best_acc = 0.0
    history = {'epoch': [], 'train_loss': [], 'test_loss': [], 'test_acc': []}

    for epoch in range(config.epochs):
        print(f"\n--- Epoch {epoch + 1}/{config.epochs} ---")
        train_loss = train_one_epoch(model, trainloader, criterion, optimizer, config.device)
        test_loss, test_acc = evaluate(model, testloader, criterion, config.device)
        scheduler.step()

        history['epoch'].append(epoch + 1)
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)

        if test_acc > best_acc:
            print(f"新的最佳准确率: {test_acc:.2f}% (优于之前的 {best_acc:.2f}%)。正在保存模型...")
            best_acc = test_acc
            model_path = os.path.join(config.model_save_path, f"{model_name}_best.pth")
            torch.save(model.state_dict(), model_path)

        print(
            f"Epoch {epoch + 1} Summary: Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

    end_time = time.time()
    total_time_min = (end_time - start_time) / 60

    print(f"\n{'=' * 40}")
    print(f" 实验结束: {model_name}")
    print(f"  -> 最高测试准确率: {best_acc:.2f}%")
    print(f"  -> 总训练时间: {total_time_min:.2f} 分钟")

    log_df = pd.DataFrame(history)
    log_filename = os.path.join(config.log_save_path, f"{model_name}_log.csv")
    log_df.to_csv(log_filename, index=False)
    print(f"  -> 训练日志已保存到: {log_filename}")

    summary_data = {
        'model_name': [model_name],
        'best_accuracy': [f"{best_acc:.2f}"],
        'params_M': [f"{params_m:.2f}"],
        'flops_G': [f"{flops_g:.2f}"],
        'training_time_min': [f"{total_time_min:.2f}"]
    }
    summary_df = pd.DataFrame(summary_data)
    summary_filename = os.path.join(config.results_path, "summary_results.csv")

    if not os.path.exists(summary_filename):
        summary_df.to_csv(summary_filename, index=False)
    else:
        summary_df.to_csv(summary_filename, mode='a', header=False, index=False)
    print(f"  -> 最终结果已汇总到: {summary_filename}")
    print(f"{'=' * 40}\n")

In [16]:
# 7. 可视化与保存对比图


def plot_and_save_comparison_chart(experiment_names, metric, config):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 8))

    for name in experiment_names:
        log_file = os.path.join(config.log_save_path, f"{name}_log.csv")
        if not os.path.exists(log_file):
            print(f"警告: 未找到日志文件 {log_file}，跳过该实验。")
            continue
        df = pd.read_csv(log_file)
        ax.plot(df['epoch'], df[metric], label=name, marker='o', linestyle='-', markersize=4)

    metric_title = metric.replace('_', ' ').title()
    ax.set_title(f'{metric_title} Comparison', fontsize=16, fontweight='bold')
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel(metric_title, fontsize=12)
    ax.legend(fontsize=10, loc='best')
    ax.grid(True)

    plot_save_path = os.path.join(config.results_path, "plots")
    os.makedirs(plot_save_path, exist_ok=True)

    plot_filename = f"comparison_{'_vs_'.join(experiment_names)}_{metric}.png"
    full_save_path = os.path.join(plot_save_path, plot_filename)

    plt.savefig(full_save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"对比图已保存到: {full_save_path}")

In [17]:
if __name__ == '__main__':
    experiments = {
        "1_CustomCNN_Plain_No-BN": lambda: CustomCNN(use_residual=False, use_bn=False),
        "2_CustomCNN_Plain_With-BN": lambda: CustomCNN(use_residual=False, use_bn=True),
        "3_CustomCNN_Residual_With-BN": lambda: CustomCNN(use_residual=True, use_bn=True),
        "4_PlainNet-18": lambda: ResNet18(use_bn=True, use_residual=False),
        "5_ResNet-18_Standard": lambda: ResNet18(use_bn=True, use_residual=True),
        "6_ResNet-18_No-BN": lambda: ResNet18(use_bn=False, use_residual=True),
    }

    for name, model_function in experiments.items():
        run_experiment(name, model_function)


开始实验: 1_CustomCNN_Plain_No-BN
模型: 1_CustomCNN_Plain_No-BN
  -> 参数量 (Params): 0.45 M
  -> 计算量 (FLOPs): 0.04 G

--- Epoch 1/50 ---


新的最佳准确率: 42.09% (优于之前的 0.00%)。正在保存模型...
Epoch 1 Summary: Train Loss: 1.8442 | Test Loss: 1.5328 | Test Acc: 42.09%

--- Epoch 2/50 ---


新的最佳准确率: 54.20% (优于之前的 42.09%)。正在保存模型...
Epoch 2 Summary: Train Loss: 1.4041 | Test Loss: 1.2517 | Test Acc: 54.20%

--- Epoch 3/50 ---


新的最佳准确率: 59.89% (优于之前的 54.20%)。正在保存模型...
Epoch 3 Summary: Train Loss: 1.1843 | Test Loss: 1.1190 | Test Acc: 59.89%

--- Epoch 4/50 ---


新的最佳准确率: 65.54% (优于之前的 59.89%)。正在保存模型...
Epoch 4 Summary: Train Loss: 1.0368 | Test Loss: 0.9947 | Test Acc: 65.54%

--- Epoch 5/50 ---


...

Epoch 46 Summary: Train Loss: 0.0643 | Test Loss: 0.5432 | Test Acc: 88.42%

--- Epoch 47/50 ---


Epoch 47 Summary: Train Loss: 0.0625 | Test Loss: 0.5462 | Test Acc: 88.26%

--- Epoch 48/50 ---


Epoch 48 Summary: Train Loss: 0.0598 | Test Loss: 0.5501 | Test Acc: 88.42%

--- Epoch 49/50 ---


Epoch 49 Summary: Train Loss: 0.0599 | Test Loss: 0.5491 | Test Acc: 88.40%

--- Epoch 50/50 ---


Epoch 50 Summary: Train Loss: 0.0596 | Test Loss: 0.5494 | Test Acc: 88.40%

 实验结束: 6_ResNet-18_No-BN
  -> 最高测试准确率: 88.47%
  -> 总训练时间: 35.04 分钟
  -> 训练日志已保存到: results/training_logs/6_ResNet-18_No-BN_log.csv
  -> 最终结果已汇总到: results/summary_results.csv



In [18]:

    print("\n\n" + "=" * 50)
    print("开始生成对比图...")
    print("=" * 50)

    # --- 对比BN对CustomCNN模型的影响 ---
    plot_and_save_comparison_chart(
        experiment_names=['1_CustomCNN_Plain_No-BN', '2_CustomCNN_Plain_With-BN'],
        metric='test_loss',
        config=config
    )

    # --- 对比残差连接对CustomCNN模型的影响 ---
    plot_and_save_comparison_chart(
        experiment_names=['2_CustomCNN_Plain_With-BN', '3_CustomCNN_Residual_With-BN'],
        metric='test_acc',
        config=config
    )

    # --- 对比残差连接对深层网络(ResNet)的影响 ---
    plot_and_save_comparison_chart(
        experiment_names=['4_PlainNet-18', '5_ResNet-18_Standard'],
        metric='test_loss',
        config=config
    )

    # --- 对比BN对深层网络(ResNet)的影响 ---
    plot_and_save_comparison_chart(
        experiment_names=['5_ResNet-18_Standard', '6_ResNet-18_No-BN'],
        metric='test_acc',
        config=config
    )

    # --- 对比所有模型的测试准确率 ---
    plot_and_save_comparison_chart(
        experiment_names=list(experiments.keys()),
        metric='test_acc',
        config=config
    )



开始生成对比图...
对比图已保存到: results/plots/comparison_1_CustomCNN_Plain_No-BN_vs_2_CustomCNN_Plain_With-BN_test_loss.png
对比图已保存到: results/plots/comparison_2_CustomCNN_Plain_With-BN_vs_3_CustomCNN_Residual_With-BN_test_acc.png
对比图已保存到: results/plots/comparison_4_PlainNet-18_vs_5_ResNet-18_Standard_test_loss.png
对比图已保存到: results/plots/comparison_5_ResNet-18_Standard_vs_6_ResNet-18_No-BN_test_acc.png
对比图已保存到: results/plots/comparison_1_CustomCNN_Plain_No-BN_vs_2_CustomCNN_Plain_With-BN_vs_3_CustomCNN_Residual_With-BN_vs_4_PlainNet-18_vs_5_ResNet-18_Standard_vs_6_ResNet-18_No-BN_test_acc.png
